In [1]:
#pip install import-ipynb
import import_ipynb

In [2]:
from web_scrapper import web_scrapper
from Boolean_search import boolean_search
from scores import retrieved_relevant_to_precision_recall
from scores import f_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Web Scrapping and Indexing #

Πρώτα θα διαλέξω ποιούς συνδέσμους θέλω να έχω στο ευρετήριο.

In [5]:
web_scrapper()

Choose an option:
1. Enter URLs manually.
2. Read URLs from links.txt.
Type 'STOP' to stop the scraping and save the data.


Enter your choice (1/2):  1
Enter URL (or type 'STOP' to stop):  https://en.wikipedia.org/wiki/Bilinear_interpolation


Data for Bilinear interpolation added to the collection.
Saving data.
Saving data to all_wikipedia_data.json
Saving data to all_wikipedia_index.json


Enter URL (or type 'STOP' to stop):  https://en.wikipedia.org/wiki/Quadrilateral


Data for Quadrilateral added to the collection.
Saving data.
Saving data to all_wikipedia_data.json
Saving data to all_wikipedia_index.json


Enter URL (or type 'STOP' to stop):  https://en.wikipedia.org/wiki/Euclidean_geometry


Data for Euclidean geometry added to the collection.
Saving data.
Saving data to all_wikipedia_data.json
Saving data to all_wikipedia_index.json


Enter URL (or type 'STOP' to stop):  stop


Stopping.


Το ευρετήριο αποθηκεύεται στο σύστημα διότι είναι δύσκολο να επαναϋπολογίζεται.

# Boolean Search

Η πρώτη μέθοδος που θα ελέγξουμε είναι η boolean αναζήτηση.

Η πρόταση που θα διαλέξουμε για αυτή είναι:

In [10]:
boolsent = "love and not mathematics or not 10 and two or bilinear" 

Την διαλέξαμε αυτή την πρόταση διότι έχει λέξεις που δεν ανήκουν και είναι περίπλοκη.

Ήδη γνωρίζουμε ότι το αποτέλεσμα πρέπει να είναι [0, 1], για να δούμε τι μας λέει η συνάρτηση.

In [13]:
retrieved = boolean_search(boolsent)
retrieved

[0, 1]

Επειδή είναι μαθηματικά συνόλων δεν μπορεί να αστοχήσει εύκολα.

Για να ελέγχουμε την ακρίβεια και ανάκληση της απάντησης.

In [16]:
relevant = [0, 1]

In [17]:
retrieved_relevant_to_precision_recall(retrieved, relevant)

(1.0, 1.0)

Το πρώτο είναι η ακρίβεια και το δεύτερο η ανάκληση και η συνάρτηση τα πήγε καλά
για να δούμε και το f score.

In [19]:
f_score(retrieved, relevant)

1.0

Και εδώ βαθμολογήθηκε το μέγιστο.

# Vector Space Model #

Τώρα θα δούμε την μέθοδο διανυσμάτων VSM.

In [23]:
query = "blah blah blah mathematics two love expressed obtained values"

Επιλέξαμε αυτή την φράση διότι είναι περιπλοκή και έχει λέξεις που δεν υπάρχουν στο ευρετήριο.

In [25]:
from tf_idf import load_index
from tf_idf import load_data
index = load_index()
docum = load_data()

Χρειαζόμαστε το ευρετήριο και τα κείμενα για υπολογίσουμε το διάνυσμα.

In [27]:
from vector_space_model import cosine_similarity
retrieved = cosine_similarity(query, index, docum)

Saving data to all_wikipedia_query.json
Cosine Similarity:
 [0.02583278 0.00749943 0.00704627]


Με αυτό το αποτέλεσμα καταλαβαίνουμε ότι το πιο σχετικό είναι το πρώτο έγγραφο,
αλλά ξέρουμε ότι τα έγραφα που πρέπει να επιστρέψει είναι το πρώτο και το δεύτερο.
Για να δούμε πια επίπτωση θα έχει στην βαθμολόγηση του.

In [29]:
retriev = list();
for i, entry in enumerate(retrieved):
    if entry > 0.01:
        retriev.append(i)
retriev

[0]

In [30]:
retrieved_relevant_to_precision_recall(retriev, relevant)

(1.0, 0.5)

Αφού το ένα αρχείο που επέστρεψε είναι σωστό σαν ακρίβεια είναι σωστό,
αλλά αφού δεν έφερε το άλλο σωστό έγγραφο, η ανάκληση είναι η μισή.

In [32]:
f_score(retriev, relevant)

0.6666666666666666

Άρα λόγω τις αβεβαιότητας, μπορούμε να διαλέξουμε το όριο, το οποίο επηρεάζει και τα αποτελέσματα.

# Propapilistic Model

Για αυτή την μέθοδο χρησιμοποιήσαμε την Okapi BM25, εφόσον είναι γνωστή και αξιόπιστη,
με τις μεταβλητές που λειτουργούν καλυτέρα.

In [36]:
from okabi import okabi_search
retrieved = okabi_search(index, docum, query)
retrieved

{0, 1, 2}

Φαίνεται να έχει καλή ανάκληση αλλά κακή ακρίβεια.

In [38]:
retrieved_relevant_to_precision_recall(retrieved, relevant)

(0.6666666666666666, 1.0)

In [39]:
f_score(retrieved, relevant)

0.8

Επειδή η απάντηση έχει λάθη η βαθμολογία του είναι μειωμένη.